In [14]:
####
#Clean Porn Dataset
#Tianhao
#2021/7/29
####
import pandas as pd
import datetime


In [9]:
### Read xlsx File
March_7 = pd.read_excel(r'C:\Users\raymo\Documents\GitHub\Callander-Research\Porn database\Data\RAW\PH_gay\2021-03-07.xlsx')


In [28]:
March_7["Extract Date"] = datetime.datetime(2021,3,7)
March_7["tags"][1].split(" ")

['Tags',
 '\n',
 'big',
 'cock',
 'anal',
 'gay',
 'glass',
 'dildo',
 'eye',
 'rolling',
 'big',
 'dildo',
 'creampie',
 'gay',
 'cumshot',
 'gay',
 'hd',
 '1080',
 'big',
 'ass',
 'bubble',
 'butt',
 'chaturbate',
 'muscle',
 'bigdick',
 '\n',
 'Suggest']